# Демо: Семантический поиск заведений 2GIS

Этот ноутбук демонстрирует работу семантического поиска заведений.

## Перед запуском

Убедитесь, что вы выполнили подготовку данных:
```bash
python step1_prepare.py
```

## 1. Импорты и инициализация

In [ ]:
import sys
import warnings
warnings.filterwarnings('ignore')

from step2_search import SemanticSearch
import pandas as pd

# Настройки pandas для красивого вывода
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.width', 1000)

In [ ]:
# Инициализация поисковой системы (займет 3-5 секунд)
print("Загрузка поисковой системы...")
search = SemanticSearch()
print("\n✅ Готово к поиску!")

## 2. Базовый поиск

In [ ]:
# Пример 1: Уютное кафе
query = "уютное кафе с вкусным кофе"
print(f"🔍 Запрос: '{query}'\n")

results = search.search_places(
    query=query,
    top_k=10,
    min_reviews=3,
    aggregation='weighted'
)

print(f"✅ Найдено {len(results)} заведений\n")
results[['name', 'address', 'rating', 'final_score', 'review_count']].head()

### Детальный вывод топ-3

In [ ]:
for idx, row in results.head(3).iterrows():
    print("="*80)
    print(f"{idx+1}. {row['name']}")
    print(f"   📍 {row.get('address', 'N/A')}")
    if 'category' in row:
        print(f"   🏷️  {row['category']}")
    if 'rating' in row and pd.notna(row['rating']):
        print(f"   ⭐ Рейтинг: {row['rating']:.1f}/5.0")
    print(f"   🎯 Релевантность: {row['final_score']:.3f}")
    print(f"   💬 Релевантных отзывов: {int(row['review_count'])}")
    
    # Показываем релевантные отзывы
    print("\n   💭 Релевантные отзывы:")
    highlights = search.get_place_highlights(row['place_id'], query, top_k=2)
    for i, review in enumerate(highlights, 1):
        print(f"      {i}. {review[:200]}...")
    print()

## 3. Сравнение методов агрегации

In [ ]:
query = "недорогой ресторан с большими порциями"
print(f"🔍 Запрос: '{query}'\n")

for aggregation in ['weighted', 'mean', 'max']:
    results = search.search_places(
        query=query,
        top_k=5,
        aggregation=aggregation
    )
    
    print(f"\n{'='*80}")
    print(f"Метод: {aggregation.upper()}")
    print("="*80)
    
    for idx, row in results.iterrows():
        print(f"{idx+1}. {row['name'][:50]:50} | Score: {row['final_score']:.3f} | Отзывов: {int(row['review_count'])}")

## 4. Различные типы запросов

In [ ]:
queries = [
    "тихое место для работы с wifi",
    "романтическое место для свидания",
    "заведение с живой музыкой",
    "семейное кафе с детской зоной",
    "модный бар с коктейлями",
]

for query in queries:
    results = search.search_places(query, top_k=3)
    
    print(f"\n{'='*80}")
    print(f"🔍 {query}")
    print("="*80)
    
    if len(results) > 0:
        for idx, row in results.iterrows():
            print(f"{idx+1}. {row['name']} - Релевантность: {row['final_score']:.3f}")
    else:
        print("❌ Нет результатов")

## 5. Анализ релевантных отзывов

In [ ]:
# Поиск релевантных отзывов (без группировки по заведениям)
query = "вкусный кофе"
print(f"🔍 Запрос: '{query}'\n")

relevant_reviews = search.search_reviews(query, top_k=20)

print(f"✅ Найдено {len(relevant_reviews)} релевантных отзывов\n")

# Показываем топ-10
for idx, row in relevant_reviews.head(10).iterrows():
    print(f"{idx+1}. Score: {row['similarity_score']:.3f}")
    print(f"   {row[search.text_column][:150]}...\n")

## 6. Интерактивный поиск

Запустите эту ячейку и вводите свои запросы!

In [ ]:
def interactive_search_notebook():
    """Интерактивный поиск в ноутбуке"""
    print("💡 Введите запрос (или 'exit' для выхода)")
    print("\nПримеры:")
    print("  - уютное кафе с вкусным кофе")
    print("  - недорогой ресторан с большими порциями")
    print("  - тихое место для работы")
    print("\n" + "="*80)
    
    while True:
        query = input("\n🔍 Ваш запрос: ").strip()
        
        if not query or query.lower() in ['exit', 'quit', 'выход']:
            print("\n👋 До свидания!")
            break
        
        results = search.search_places(query, top_k=5)
        
        if len(results) == 0:
            print("❌ Ничего не найдено. Попробуйте другой запрос.")
            continue
        
        print(f"\n✅ Найдено {len(results)} заведений:\n")
        
        for idx, row in results.iterrows():
            print(f"{idx+1}. {row['name']}")
            print(f"   📍 {row.get('address', 'N/A')}")
            print(f"   🎯 Релевантность: {row['final_score']:.3f}")
            print(f"   💬 Релевантных отзывов: {int(row['review_count'])}")
            print()

# Запустите интерактивный режим
interactive_search_notebook()

## 7. Экспорт результатов

In [ ]:
# Сохранение результатов поиска в CSV
query = "уютное кафе с вкусным кофе"
results = search.search_places(query, top_k=20)

output_file = f"search_results_{query[:30].replace(' ', '_')}.csv"
results.to_csv(output_file, index=False, encoding='utf-8')

print(f"✅ Результаты сохранены в: {output_file}")

## 8. Статистика и метрики

In [ ]:
print("📊 Статистика базы данных:\n")
print(f"Всего отзывов: {len(search.reviews_df):,}")
print(f"Всего заведений: {len(search.places_df):,}")
print(f"Векторов в индексе: {search.index.ntotal:,}")
print(f"\nМодель: {search.model}")
print(f"Размерность эмбеддингов: {search.model.get_sentence_embedding_dimension()}")

## Готово! 🎉

Теперь вы можете:
- Экспериментировать с разными запросами
- Пробовать разные методы агрегации
- Анализировать релевантные отзывы
- Настраивать параметры в `config.py`

Для более подробной информации смотрите [README.md](README.md)